## testes de IA

In [7]:
# --- Istalar depedencias (o break sistem e apenas o meu caso, acredito) ---
!pip install PyPDF2 google-generativeai python-dotenv --break-system-packages
!pip install --upgrade google-generativeai --break-system-packages
!pip install --upgrade google-generativeai PyPDF2 --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [10]:

import PyPDF2
import google.generativeai as genai
from IPython.display import display, Markdown
import os 

API_KEY = "" #esta funcionando, gerar outra chave quando for testar/ mostrar

if API_KEY == "SUA_API_KEY_AQUI":
    raise ValueError(" ERRO: Insira sua API Key real do Google AI Studio!")

genai.configure(api_key=API_KEY)

# Use 'gemini-2.0-flash-exp' para estabilidade, ou 'gemini-2.5-flash-preview-0514' para o mais novo
model = genai.GenerativeModel('gemini-2.0-flash-exp')  

# --- FUNÇÃO PARA LER O PDF ---
def extrair_texto_pdf(caminho_pdf):
    if not os.path.exists(caminho_pdf):
        raise FileNotFoundError(f"  Arquivo '{caminho_pdf}' não encontrado! Coloque o PDF na pasta do notebook.")
    
    texto_completo = ""
    with open(caminho_pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num, page in enumerate(reader.pages, 1):
            texto = page.extract_text()
            if texto:  # Ignora páginas vazias
                texto_completo += f"\n--- PÁGINA {page_num} ---\n{texto}\n"
    return texto_completo.strip()

# --- CARREGANDO O DOCUMENTO ---
print("📄 Carregando aula08.pdf...")
try:
    documento = extrair_texto_pdf("aula08.pdf")
    print(f" Documento carregado! {len(documento)} caracteres extraídos.")
except Exception as e:
    print(f"Erro ao ler PDF: {e}")
    documento = ""  # Fallback vazio

# --- FUNÇÃO DO CHATBOT (RAG BÁSICO) ---
def perguntar_sobre_documento(pergunta):
    if not documento:
        return "Erro: Documento não carregado. Verifique o PDF."
    
    # Prompt otimizado para RAG (Retrieval-Augmented Generation)
    prompt = f"""
    Você é um assistente especializado na AULA 08 de Inteligência Artificial (Projeto Final).
    Responda APENAS com base no CONTEÚDO do documento abaixo. 
    Seja claro, objetivo e use português brasileiro.
    Se a pergunta não puder ser respondida só com o documento, diga: "Não encontrei essa informação no documento fornecido."
    
    === CONTEÚDO DO DOCUMENTO ===
    {documento[:8000]}  # Limite para evitar overflow (o Gemini lida bem com isso)
    === FIM DO DOCUMENTO ===
    
    Pergunta do usuário: {pergunta}
    
    Resposta:
    """
    
    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"Erro na API Gemini: {str(e)}. Verifique sua chave ou quota."

# --- INTERFACE INTERATIVA NO JUPYTER ---
def chatbot():
    display(Markdown("# 🤖 Chatbot RAG - Aula 08: Projeto Final de IA"))
    display(Markdown("**Faça perguntas sobre o PDF! Ex: 'Qual a data de entrega?'**\n"
                     "*Digite 'sair' para encerrar.*"))
    
    while True:
        try:
            pergunta = input("\n**Você:** ").strip()
            if not pergunta:
                continue
            if pergunta.lower() in ['sair', 'exit', 'quit', 'sair']:
                display(Markdown("** encerrado.**"))
                break
            
            print("Pensando...")
            resposta = perguntar_sobre_documento(pergunta)
            display(Markdown(f"**Chatbot:** {resposta}"))
            print("\n" + "="*50) 
        except KeyboardInterrupt:
            display(Markdown("**Sessão interrompida pelo usuário.**"))
            break
        except Exception as e:
            display(Markdown(f"**Erro inesperado: {str(e)}**"))

if __name__ == "__main__":
    chatbot()

📄 Carregando aula08.pdf...
✅ Documento carregado! 7435 caracteres extraídos.


# 🤖 Chatbot RAG - Aula 08: Projeto Final de IA

**Faça perguntas sobre o PDF! Ex: 'Qual a data de entrega?'**
*Digite 'sair' para encerrar.*


**Você:**  quais sao as 2 datas de entrega?


🤔 Chatbot: Pensando...


...


**Você:**  poderia sair do escopo do documento?


🤔 Chatbot: Pensando...


**Chatbot:** Não encontrei essa informação no documento fornecido.


**Você:**  sair


**👋 Chatbot encerrado. Boa sorte no projeto!**

In [ ]:
sau